# Import

In [2]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm


# Fixed RandomSeed & Setting Hyperparameter

In [3]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [4]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
DEVICE

device(type='cuda')

# Data Load

In [6]:
train = pd.read_csv('../data/train/train.csv')

# Define Model

In [7]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)

# Train

In [ ]:
def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(X_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=1, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models

In [ ]:
# 학습
trained_models = train_lstm(train)

Training LSTM:   0%|          | 0/193 [00:00<?, ?it/s]C:\Users\bae\AppData\Local\Temp\ipykernel_4896\2939504505.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  X_train = torch.tensor(X_train).float().to(DEVICE)
Training LSTM: 100%|██████████| 193/193 [24:45<00:00,  7.70s/it]


# Prediction

In [ ]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted['매출수량'].values[-LOOKBACK:]
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals.reshape(-1, 1))
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 1))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val
            })

    return pd.DataFrame(results)


In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('../data/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)
    
full_pred_df = pd.concat(all_preds, ignore_index=True)

c:\Users\bae\anaconda3\envs\py310-aimers\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\bae\anaconda3\envs\py310-aimers\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\bae\anaconda3\envs\py310-aimers\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\bae\anaconda3\envs\py310-aimers\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
c:\Users\bae\anaconda3\envs\py310-aimers\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted w

In [25]:
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,"(느티나무 셀프BBQ_1인 수저세트,)",6.719441
1,TEST_00+2일,"(느티나무 셀프BBQ_1인 수저세트,)",3.162586
2,TEST_00+3일,"(느티나무 셀프BBQ_1인 수저세트,)",1.369801
3,TEST_00+4일,"(느티나무 셀프BBQ_1인 수저세트,)",2.719870
4,TEST_00+5일,"(느티나무 셀프BBQ_1인 수저세트,)",5.801590
...,...,...,...
13505,TEST_09+3일,"(화담숲카페_현미뻥스크림,)",20.108158
13506,TEST_09+4일,"(화담숲카페_현미뻥스크림,)",16.663351
13507,TEST_09+5일,"(화담숲카페_현미뻥스크림,)",24.480161
13508,TEST_09+6일,"(화담숲카페_현미뻥스크림,)",27.790758


# Submission

In [33]:
# def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
#     # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
#     pred_dict = dict(zip(
#         zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
#         pred_df['매출수량']
#     ))

#     final_df = sample_submission.copy()

#     for row_idx in final_df.index:
#         date = final_df.loc[row_idx, '영업일자']
#         for col in final_df.columns[1:]:  # 메뉴명들
#             final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

#     return final_df

def convert_to_submission_format(pred_df: pd.DataFrame,
                                 sample_submission: pd.DataFrame) -> pd.DataFrame:
    pred_df = pred_df.copy()
    
    # 1) 튜플이면 문자열로 변환
    pred_df['영업장명_메뉴명'] = pred_df['영업장명_메뉴명'].apply(
        lambda x: x[0] if isinstance(x, tuple) else x
    )
    
    # 2) (영업일자 × 메뉴) wide 형태로 변환
    wide = (
        pred_df
        .pivot_table(index='영업일자',
                     columns='영업장명_메뉴명',
                     values='매출수량',
                     aggfunc='sum')   # 혹시 중복 있으면 합산
        .astype(float)
    )

    # 3) sample_submission 형태에 맞춰 align
    final_df = sample_submission.copy()
    final_df = final_df.set_index('영업일자')

    aligned = (
        wide.reindex(index=final_df.index, fill_value=0)
            .reindex(columns=final_df.columns, fill_value=0)
    )

    final_df = final_df.astype({col: float for col in final_df.columns if col != '영업일자'})
    final_df.loc[:, :] = aligned.values
    final_df = final_df.reset_index()

    return final_df


In [34]:
sample_submission = pd.read_csv('../data/sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('../baseline_submission.csv', index=False, encoding='utf-8-sig')

In [35]:
submission

,영업일자,느티나무 셀프BBQ_1인 수저세트,느티나무 셀프BBQ_BBQ55(단체),"느티나무 셀프BBQ_대여료 30,000원","느티나무 셀프BBQ_대여료 60,000원","느티나무 셀프BBQ_대여료 90,000원","느티나무 셀프BBQ_본삼겹 (단품,실내)",느티나무 셀프BBQ_스프라이트 (단체),느티나무 셀프BBQ_신라면,느티나무 셀프BBQ_쌈야채세트,...,화담숲주막_스프라이트,화담숲주막_참살이 막걸리,화담숲주막_찹쌀식혜,화담숲주막_콜라,화담숲주막_해물파전,화담숲카페_메밀미숫가루,화담숲카페_아메리카노 HOT,화담숲카페_아메리카노 ICE,화담숲카페_카페라떼 ICE,화담숲카페_현미뻥스크림
0,TEST_00+1일,6.719441,0.000000,7.740270,3.501088,0.523453,1.541909,2.369279,2.344821,3.072474,...,9.244626,17.921509,14.991831,7.855288,70.462128,38.402624,0.000000,29.410436,12.016371,23.918862
1,TEST_00+2일,3.162586,12.006579,2.223160,0.546116,0.000000,1.223568,4.464143,2.467271,1.676339,...,6.837491,6.414076,6.537210,6.606168,33.652418,25.100696,0.000000,20.296730,10.277245,4.345945
2,TEST_00+3일,1.369801,17.489786,4.004414,0.927059,0.000000,1.269181,5.731761,2.937773,1.441989,...,5.476991,0.000000,2.562306,6.385094,23.871394,26.703556,0.000000,13.904313,7.063367,0.000000
3,TEST_00+4일,2.719870,32.337138,2.310801,1.696748,0.188710,1.265622,16.428719,3.642769,1.651352,...,7.272205,0.000000,3.812046,9.514292,41.740298,18.751561,0.000000,17.322765,7.991291,2.271307
4,TEST_00+5일,5.801590,61.897304,3.013414,1.222601,0.198258,1.329626,18.100113,5.372316,2.779173,...,8.673743,8.191969,10.735761,13.653431,55.184022,20.162074,0.000000,19.162111,7.051322,8.919431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TEST_09+3일,1.388723,36.672949,3.237364,0.995804,0.000000,1.363778,8.913214,1.085746,1.326308,...,3.867839,2.057021,5.662376,5.016408,54.838841,18.347310,37.595410,58.651609,16.048447,20.108158
66,TEST_09+4일,0.647931,71.705295,2.444685,1.814162,0.211964,1.196497,19.049849,1.008227,1.191607,...,4.425293,3.123244,0.000000,5.749790,67.621584,20.535055,45.006449,52.607975,14.179467,16.663351
67,TEST_09+5일,1.625794,75.392240,3.933325,1.231137,0.142626,1.297823,21.797493,0.771402,1.807707,...,5.282610,10.139281,3.503247,8.498251,72.234527,21.299973,53.782386,49.938631,15.195310,24.480161
68,TEST_09+6일,4.516461,23.798159,8.970833,3.140051,0.201186,1.277381,13.028440,2.130589,3.037107,...,7.521313,21.237213,8.382307,11.919602,108.916494,21.615222,49.764193,51.861414,17.311364,27.790758
